## Song Recommendation (Gradient Descent Menthods)
- $P$ is the user-factor matrix (u $x$ k)
- $Q$ is the song-factor matrix (s $x$ k)
- $R$ is the user-song matrix (u $x$ s)
- $M$ is the preprocessed/ready-to-use triplet matrix (n $x$ 3)
- $n$ is the number of lines fetched from the file
- $u$ is the number of users
- $s$ is the number of songs

In [1]:
import time
import pandas as pd
import numpy.linalg as la
import numpy as np
import os
import scipy.sparse as sp
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(precision=3)

## Parameter Configuration Panel

In [2]:
path = "/Users/oeken/Downloads/train_triplets.txt"
n = 300000
n_test = 100     # size of the test set
epochs = 10      # how many times to go over the whole dataset?
delta = 0.01     # learning rate
lambda1 = 0.0   # regularizer coeff for p
lambda2 = 0.0   # regularizer coeff for q
S = 1          # batch size
k = 10           # number of latent factors

## Helper Funtions Level 2

In [3]:
# finds the corresponding bin value for a given number x 
def count_to_bin(a,b):
    bin = 0
    for x in range (b):
        if a>=2**x and a<=((2**(x+1))-1):
            return x+1
    return b

# assigns incremental IDs and creates 2-way dictionaries
def create_dictionary(list):
    dict_1 = {}
    dict_2 = {}
    count = 0;
    for element in list:
        if not element in dict_1:
            dict_1[element] = count
            dict_2[count] = element
            count = count+1
    return dict_1, dict_2

## Helper Functions Level 1

In [4]:
# reads n triplets from given file and returns them in a list
def read_triplets(path, n):
    with open(path) as infile:
        count = 0;
        triplets = []        
        for line in infile:
            temp_list=[]
            count = count+1
            temp_list = line.split()
            triplets.append(temp_list)
            if count==n:
                break        
    return triplets


# removes triplets containing entities 5 or less occurrences
def elim_five_or_less(triplets):
    
    while 1 : 
        min_listeners = triplets.groupby('users').count().min()[0]
        min_songs = triplets.groupby('songs').count().min()[0]
        triplets = triplets.groupby('users').filter(lambda x: len(x) > 5)
        triplets = triplets.groupby('songs').filter(lambda x: len(x) > 5)
        if min_listeners>5 and min_songs>5:
            break;
    return triplets


# assigns ids to users and songs and binifies the play counts
def get_ids_and_bins(M_raw):
    M = M_raw.copy()
    users = M[:,0] #first column represents the users
    songs = M[:,1] #second column represents the songs
    ud1, ud2 = create_dictionary(users) #gives incremental integer values to all users
    sd1, sd2 = create_dictionary(songs) #gives incremental integer values to all songs

    M[:,2] = [count_to_bin(x , 7) for x in M[:,2]] #bin representation of the counts
    M[:,1] = [sd1[x] for x in M[:,1]] #incremental represantation of the songs
    M[:,0] = [ud1[x] for x in M[:,0]] #incremental represantation of the listeners
    return M, ud1, ud2, sd1, sd2


# given an instance of p finds associated q's with it, and vice-versa. makes use of dicts
def assoc_pq(M):
    p_to_q = dict()    
    q_to_p = dict()
    for row in M:
        if row[0] in p_to_q:
            tup = p_to_q[row[0]]
            tup[0].append(row[1])
            tup[1].append(row[2])
        else:
            p_to_q[row[0]] = [[row[1]], [row[2]]]
            
        if row[1] in q_to_p:
            tup = q_to_p[row[1]]
            tup[0].append(row[0])
            tup[1].append(row[2])
        else:
            q_to_p[row[1]] = [[row[0]], [row[2]]]                    
    return p_to_q, q_to_p

def error(P, Q, M):
    e = 0
    for row in M:
        e += (row[2] - P[row[0],:].dot(Q[row[1],:]))**2        
    e += lambda1 * np.linalg.norm(P)
    e += lambda2 * np.linalg.norm(Q)
    return e, e/M.shape[0]

def rmse(P, Q, M):
    r = 0
    for row in M:
        r += (row[2] - P[row[0],:].dot(Q[row[1],:]))**2
    r /= M.shape[0]
    r = np.sqrt(r)
    return r
    

# Driver Program

### 1. Read $n$ triplets from the given file

In [5]:
triplets = read_triplets(path, n)
triplets = pd.DataFrame(triplets, columns=['users','songs','play_counts'])
triplets.head(5)

,users,songs,play_counts
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [6]:
print(triplets.shape)

(300000, 3)


### 2. Recursively eliminate triplets containing users/songs whose occurrence is less than 5

In [7]:
triplets_eliminated = elim_five_or_less(triplets)
triplets_eliminated.head(5)

,users,songs,play_counts
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
10,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1


In [8]:
triplets_eliminated.shape

(171225, 3)

### 3. Assign IDs to users/songs and binify the play counts

In [9]:
M_raw = np.asarray(triplets_eliminated) #convert list to ndarray
M_raw[:,2] = list(map(int, M_raw[:,2])) #parse the string counts to integer

M, ud1, ud2, sd1, sd2 = get_ids_and_bins(M_raw)
u = len(ud1)  # number of users
s = len(sd1)  # number of songs
n_new = len(M)
print('# of triplets: %d' % (n_new))
print('# of users: %d' % (u))
print('# of songs: %d' % (s))
print('Total rating slots: %d' % (u*s))
print('Data ratio: %.4f' % (n_new/(u*s)))

# of triplets: 171225
# of users: 5693
# of songs: 10966
Total rating slots: 62429438
Data ratio: 0.0027


### 3.1 Synthetic setup

In [10]:
# synthetic götten uydurma data
# see --> http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/
# u = 5
# s = 4
# M = np.array([[0,0,5],
#                 [0,1,3],
#                  [0,3,1],
#                  [1,0,4],
#                  [1,3,1],
#                  [2,0,1],
#                  [2,1,1],
#                  [2,3,5],                 
#                  [3,0,1],
#                  [3,3,4],
#                  [4,1,1],
#                  [4,2,5],                                  
#                  [4,3,4]])
# n_new = 13
# print('# of triplets: %d' % (n_new))
# print('# of users: %d' % (u))
# print('# of songs: %d' % (s))
# print('Total rating slots: %d' % (u*s))
# print('Data ratio: %.4f' % (n_new/(u*s)))

### 4. Seperate data into training and test set

In [11]:
np.random.seed(100)
all_indices = np.arange(n_new)
np.random.shuffle(all_indices)
test_indices = all_indices[:n_test]
#trai_indices = all_indices[n_test:]
trai_indices = all_indices # uncomment for synthetic data
print('Test set shape', test_indices.shape)
print('Trai set shape', trai_indices.shape)

Test set shape (100,)
Trai set shape (171225,)


### 5. Initialize $P$ and $Q$ matrices + Compute batch count

In [16]:
def initPQ(k):    
    np.random.seed(110)
    P = np.random.rand(u,k) # init P
    Q = np.random.rand(s,k) # init Q

    print('P shape: ', P.shape)
    print('Q shape: ', Q.shape)
    #errors = [error(P,Q,M)]
    #print('Initial error: ', errors)

    print()

    batch_count = int(n_new / S)
    print('Train count:', trai_indices.size)
    print('Batch size: %d' % S)
    print('Batch count: %d' % batch_count)
    return P, Q

### 6. Define gradient descent with given batch size, $S$
* Note that if $S = <$size of training data$>$ then the algorithm reduces to Standard Gradient Descent
* Note that if $S = 1$ then the algorithm reduces the Classical Stochastic Gradient Descent

In [17]:
def GD(epochs, P, Q, M, S):
    errors = []
    errors.append(error(P,Q,M))
    
    rmses = []
    rmses.append(rmse(P,Q,M))
    
    t = time.process_time()
    print('i. E(P,Q) = %.3f, %.3f' % errors[0])

    for e in range(epochs):  # how many times to go over the whole dataset
        t_epoch = time.process_time()
        np.random.shuffle(trai_indices)  # permute the training data
        for b in range(batch_count):  # for each batch

            batch = trai_indices[b*S:b*S+S]  # fetch the next training data indices
            M_b = M[batch]
            p_grad = {}
            q_grad = {}

            for row in M_b:  # compute gradients
                p = row[0]
                q = row[1]
                r = row[2]
                p_vec = P[p,:]
                q_vec = Q[q,:]
                term = (r - p_vec.dot(q_vec))

                if p in p_grad:
                    p_grad[p] += term * q_vec
                else:
                    p_grad[p] = term * q_vec - (lambda1 * p_vec)

                if q in q_grad:
                    q_grad[q] += term * p_vec
                else:
                    q_grad[q] = term * p_vec - (lambda2 * q_vec)


            for p in p_grad.keys():
                P[p,:] += delta * p_grad[p]

            for q in q_grad.keys():
                Q[q,:] += delta * q_grad[q]

        error_current = error(P, Q, M)
        errors.append(error_current)
        rmse_current = rmse(P,Q,M)
        rmses.append(rmse_current)
        print('%d. E(P,Q) = %.3f, %.3f -- time: %.2f' % (e, error_current[0], error_current[1], time.process_time()-t_epoch))


    elapsed_time = time.process_time() - t
    print('%d Epochs, Total seconds: %f' % (epochs, elapsed_time))
    return errors, rmses

## 1. SGD ($S = 1$)

In [18]:
epochs = 10
S = 1
k = 8

P, Q = initPQ(k)
(e1,r1) = GD(epochs, P, Q, M, S)

P shape:  (5693, 8)
Q shape:  (10966, 8)

Train count: 171225
Batch size: 1
Batch count: 171225
i. E(P,Q) = 269582.674, 1.574


NameError: name 'batch_count' is not defined

### Error Plots

In [ ]:
%matplotlib inline
plt.plot(e1)

In [ ]:
plt.plot(r1)

## 2. Modified SGD ($1<S<n$ )

In [ ]:
epochs = 10
S = 1
k = 8

P, Q = initPQ(k)
(e2,r2) = GD(epochs, P, Q, M, S)

In [ ]:
%matplotlib inline
plt.plot(e2)

In [ ]:
plt.plot(r2)

## 3. Standard GD ($S = n$)

In [ ]:
epochs = 10
S = 1
k = 8
P, Q = initPQ(k)
(e3,r3) = GD(epochs, P, Q, M, S)

In [ ]:
%matplotlib inline
plt.plot(e3)

In [ ]:
plt.plot(r3)

In [ ]:
#pd.DataFrame(P.dot(Q.T)).head(5)
# see --> https://hec.su/lWz